In [3]:
import keras
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np


In [12]:
#First will write video in mask and without - maybe will add picturest for model test
import cv2

In [13]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera!")
    exit()
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

In [14]:
frame_time = cap.get(cv2.CAP_PROP_FPS)
frame_time

30.0

In [15]:
out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width, frame_height))

In [16]:
while True:
    ret, frame = cap.read()
    if not ret:
        print("Problem!")
        break 
    
    out.write(frame)
    cv2.imshow('Camera', frame)
    if cv2.waitKey(int(frame_time)) == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

In [1]:
# Можна розбити на фото і додати до датасету, проте це мабуть недоречно. Розділю даний датасет data.zip на train, test

In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Завантаження і передобробка даних
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    r"E:TRAINING_DIRdata.zip",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    r"E:TRAINING_DIRdata.zip",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Створення моделі
model = tf.keras.models.Sequential([
    tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Компіляція моделі
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Навчання моделі
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=5
)

# Оцінка моделі
test_loss, test_acc = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

# Щось роблю не так - не розділено на 2 класи


Found 1101 images belonging to 1 classes.
Found 275 images belonging to 1 classes.
Epoch 1/5
34/34 [==============================] - 1073s 29s/step - loss: 0.0339 - accuracy: 0.9822 - val_loss: 33.7866 - val_accuracy: 0.0000e+00
Epoch 2/5
34/34 [==============================] - 885s 26s/step - loss: 5.0681e-05 - accuracy: 1.0000 - val_loss: 34.0086 - val_accuracy: 0.0000e+00
Epoch 3/5
34/34 [==============================] - 855s 25s/step - loss: 2.0401e-05 - accuracy: 1.0000 - val_loss: 32.0128 - val_accuracy: 0.0000e+00
Epoch 4/5
34/34 [==============================] - 849s 25s/step - loss: 1.5200e-05 - accuracy: 1.0000 - val_loss: 29.0731 - val_accuracy: 0.0000e+00
Epoch 5/5
8/8 [==============================] - 36s 4s/step - loss: 26.2177 - accuracy: 0.0000e+00
Test Loss: 26.21771240234375
Test Accuracy: 0.0


In [10]:
import os
os.makedirs('E:/model_masks', exist_ok=True)
model.save('E:/model_masks/model.h5')

# Завантаження навченої моделі
trained_model_path = 'E:/model_masks/model.h5'

output_file_path = 'E:/model_masks'

model_loaded=keras.models.load_model('E:/model_masks/model.h5')

In [11]:
import cv2
import numpy as np
from keras.models import load_model
model=load_model('E:/model_masks/model.h5')
results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}
rect_size = 4
cap = cv2.VideoCapture(0) 
haarcascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 
    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(224,224))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,224,224,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == ord('q'): #The Esc key
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 124ms/step
